In [1]:
import torch
import pandas as pd
from HMSDatasets  import HMSHBACSpecDataset
from model import HMSHBACSpecModel
from utils.util import set_random_seed, to_device, get_path_label, get_transforms
import pytorch_lightning as pl
import wandb
from torch import nn
import glob
from HMSDatasets import FullDataset
import numpy as np
from pl_bolts.optimizers.lr_scheduler import LinearWarmupCosineAnnealingLR

/home/ubuntu/.cache/pypoetry/virtualenvs/hmc-snuh-QMWTtYeW-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ubuntu/.cache/pypoetry/virtualenvs/hmc-snuh-QMWTtYeW-py3.10/lib/python3.10/site-packages/pl_bolts/__init__.py:11: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):
/home/ubuntu/.cache/pypoetry/virtualenvs/hmc-snuh-QMWTtYeW-py3.10/lib/python3.10/site-packages/pl_bolts/__init__.py:11: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):
/home/ubuntu/.cache/pypoetry/virtualenvs/hmc-snuh-QMWTtYeW-py3.10/lib/python3.10/site-packages/pl_bolts/models/self_supervised/amdim/amdim_module.py:34: UnderReviewWarning: The feature generate_power_seq is cur

In [2]:
import omegaconf
ori_cfg = omegaconf.OmegaConf.load("/home/ubuntu/HMC-SNUH/configs/0305.yaml")

In [3]:
torch.backends.cudnn.benchmark = True
set_random_seed(ori_cfg.seed, deterministic=ori_cfg.deterministic)
device = torch.device(ori_cfg.device)

In [4]:
from pytorch_lightning.callbacks import ModelCheckpoint
checkpoint_callback = ModelCheckpoint(
    save_top_k=10,
    monitor="loss",
    mode="min",
    dirpath="checkpoints/",
    filename="sample-mnist-{epoch:02d}-{val_loss:.2f}",
)

In [5]:
train_all = pd.read_csv("fold_dataset/train_folds.csv")

if ori_cfg.dataset == "Spectrogram":
    train_path_label, val_path_label, _, _ = get_path_label(cfg.val_fold, train_all)
    train_transform, val_transform = get_transforms(cfg)
    train_dataset = HMSHBACSpecDataset(**train_path_label, transform=train_transform)
    val_dataset = HMSHBACSpecDataset(**val_path_label, transform=val_transform)
elif ori_cfg.dataset == "EEG":
    import omegaconf
    cfg = omegaconf.OmegaConf.load("/home/ubuntu/HMC-SNUH/configs/datasets/init.yaml")
    TARGETS = ["seizure_vote", "lpd_vote", "gpd_vote", "lrda_vote", "grda_vote", "other_vote"]
    df = pd.read_csv("/data/SWB_Contribute/Data/original_data/train.csv")
    # Create a new identifier combining multiple columns
    id_cols = ['eeg_id', 'spectrogram_id', 'seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']
    df['new_id'] = df[id_cols].astype(str).agg('_'.join, axis=1)
    
    # Calculate the sum of votes for each class
    df['sum_votes'] = df[TARGETS].sum(axis=1)
    
    # Group the data by the new identifier and aggregate various features
    agg_functions = {
        'eeg_id': 'first',
        'eeg_label_offset_seconds': ['min', 'max'],
        'spectrogram_label_offset_seconds': ['min', 'max'],
        'spectrogram_id': 'first',
        'patient_id': 'first',
        'expert_consensus': 'first',
        **{col: 'sum' for col in TARGETS},
        'sum_votes': 'sum',
    }
    grouped_df = df.groupby('new_id').agg(agg_functions).reset_index()

    # Flatten the MultiIndex columns and adjust column names
    grouped_df.columns = [f"{col[0]}_{col[1]}" if col[1] else col[0] for col in grouped_df.columns]
    grouped_df.columns = grouped_df.columns.str.replace('_first', '').str.replace('_sum', '')
    
    # Normalize the class columns
    y_data = grouped_df[TARGETS].values
    y_data_normalized = y_data / y_data.sum(axis=1, keepdims=True)
    grouped_df[TARGETS] = y_data_normalized

    # Split the dataset into high and low quality based on the sum of votes
    high_quality_df = grouped_df[grouped_df['sum_votes'] >= 10].reset_index(drop=True)
    low_quality_df = grouped_df[(grouped_df['sum_votes'] < 10) & (grouped_df['sum_votes'] >= 0)].reset_index(drop=True)
    train = grouped_df

    # K fold
    from sklearn.model_selection import StratifiedGroupKFold

    train = train.groupby("spectrogram_id").head(1).reset_index(drop=True)
    sgkf = StratifiedGroupKFold(n_splits=ori_cfg.N_FOLDS, shuffle=True, random_state=ori_cfg.RANDOM_SEED)
    train["fold"] = -1

    for fold_id, (_, val_idx) in enumerate(
        sgkf.split(train, y=train["expert_consensus"], groups=train["patient_id"])
    ):
        train.loc[val_idx, "fold"] = fold_id
    
    train.to_csv('./fold_dataset/'+ "train_folds.csv", index=False)

    # load files
    raw_eegs = np.load(cfg.PRE_LOADED_RAW_EEGS, allow_pickle=True).item()
    print(f"Length of eegs: {len(raw_eegs)}")

    READ_SPEC_FILES = False
    paths_spectrograms = glob.glob(cfg.TRAIN_SPECTOGRAMS + "*.parquet")
    print(f'There are {len(paths_spectrograms)} spectrogram parquets')

    all_spectrograms = np.load(cfg.PRE_LOADED_SPECTROGRAMS, allow_pickle=True).item()

    READ_EEG_SPEC_FILES = False
    paths_eegs = glob.glob(cfg.TRAIN_EEG_SPECTROGRAM + "*.npy")
    print(f'There are {len(paths_eegs)} EEG spectrograms')

    all_eegs = np.load(cfg.PRE_LOADED_EEG_SPECTROGRAMS, allow_pickle=True).item()

    print(f"Shape of Kaggle spectrograms: {all_spectrograms[train.loc[0,'spectrogram_id']].shape}")
    print(f"Shape of EEG spectrograms: {all_eegs[train.loc[0,'eeg_id']].shape}")
    print(f"Shape of raw eegs: {raw_eegs[train.loc[0,'eeg_id']].shape}")

    # Datasets
    train_dataset = FullDataset(train[train["fold"] != ori_cfg.val_fold], 
                                eegs = raw_eegs,
                                specs = all_spectrograms,
                                eeg_specs = all_eegs,
                                mode="train")
    val_dataset = FullDataset(train[train["fold"] == ori_cfg.val_fold],
                                eegs = raw_eegs,
                                specs = all_spectrograms,
                                eeg_specs = all_eegs,
                                mode="val")

else:
    raise NotImplementedError


# val_loader = torch.utils.data.DataLoader(
#     val_dataset, batch_size=ori_cfg.batch_size, num_workers=4, shuffle=False, drop_last=False)


Length of eegs: 17089
There are 11138 spectrogram parquets
There are 17089 EEG spectrograms
Shape of Kaggle spectrograms: (300, 400)
Shape of EEG spectrograms: (128, 256, 4)
Shape of raw eegs: (10000, 8)


In [6]:
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=ori_cfg.batch_size, num_workers=16, shuffle=True, drop_last=True)

val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=ori_cfg.batch_size, num_workers=16, shuffle=False, drop_last=False)

In [7]:
class KLDivLossWithLogits(torch.nn.KLDivLoss):

    def __init__(self):
        super().__init__(reduction="batchmean")

    def forward(self, y, t):
        y = nn.functional.log_softmax(y,  dim=1)
        loss = super().forward(y, t)
        return loss

In [9]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()

        self.instance_norm = nn.InstanceNorm1d(8, affine = True)
        self.norm1a = torch.nn.LayerNorm(8)
        self.mha1 = torch.nn.MultiheadAttention(8, 4, dropout=0.1)
        self.norm1b = torch.nn.LayerNorm(8)
        self.nn1 = torch.nn.Linear(8, 8)

        self.norm2a = torch.nn.LayerNorm(8)
        self.mha2 = torch.nn.MultiheadAttention(8, 4, dropout=0.1)
        self.norm2b = torch.nn.LayerNorm(8)
        self.nn2 = torch.nn.Linear(8, 8)

        self.norm3a = torch.nn.LayerNorm(8)
        self.mha3 = torch.nn.MultiheadAttention(8, 4, dropout=0.1)
        self.norm3b = torch.nn.LayerNorm(8)
        self.nn3 = torch.nn.Linear(8, 8)

        self.layer_norm = torch.nn.LayerNorm(8)
        self.mha3 = torch.nn.MultiheadAttention(8, 8, dropout=0.1)
        self.gap = torch.nn.GlobalAveragePool1d()
    
        self.layer_norm2 = torch.nn.BatchNorm1d(10000)
        self.nn5 = torch.nn.Linear(10000, 6)

    def forward(self, input: torch.Tensor) -> torch.Tensor:
        input = torch.transpose(input, 1,2)
        input = self.instance_norm(input)
        input = torch.transpose(input, 1,2)
        x, attn_w = self.mha1(x, x, x)

        x = self.norm1b(x + input)
        x = self.nn1(x)
        x = nn.ReLU()(x)
        x2 = input+x

        x = self.norm2a(x2)
        x, attn_w = self.mha2(x, x, x)
        x = self.norm2b(x+x2)
        x = self.nn2(x)
        x = nn.ReLU()(x)
        x3 = x2+x

        x = self.norm3a(x3)
        x, attn_w = self.mha3(x, x, x)
        x = self.norm3b(x+x3)
        x = self.nn3(x)
        x = nn.ReLU()(x)

        x = self.layer_norm(x+input)
        x= self.nn4(x)
        # x = self.layer_norm2(x_n)
        # x_a, attn_w = self.mha4(x, x, x)
        # x = x_n+x_a 
        x = self.gap(x)
        x = self.layer_norm2(x)
        x = self.nn5(x)
        return x

import torch
import torch.nn as nn


class ResNet_1D_Block(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, downsampling):
        super(ResNet_1D_Block, self).__init__()
        self.bn1 = nn.BatchNorm1d(num_features=in_channels)
        self.relu = nn.ReLU(inplace=False)
        self.dropout = nn.Dropout(p=0.0, inplace=False)
        self.conv1 = nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size,
                               stride=stride, padding=padding, bias=False)
        self.bn2 = nn.BatchNorm1d(num_features=out_channels)
        self.conv2 = nn.Conv1d(in_channels=out_channels, out_channels=out_channels, kernel_size=kernel_size,
                               stride=stride, padding=padding, bias=False)
        self.maxpool = nn.MaxPool1d(kernel_size=2, stride=2, padding=0)
        self.downsampling = downsampling

    def forward(self, x):
        identity = x

        out = self.bn1(x)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.conv1(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.conv2(out)

        out = self.maxpool(out)
        identity = self.downsampling(x)

        out += identity
        return out


class EEGNet(nn.Module):

    def __init__(self, kernels, in_channels=8, fixed_kernel_size=17, num_classes=6):
        super(EEGNet, self).__init__()
        self.kernels = kernels
        self.planes = 24
        self.parallel_conv = nn.ModuleList()
        self.in_channels = in_channels
        
        for i, kernel_size in enumerate(list(self.kernels)):
            sep_conv = nn.Conv1d(in_channels=in_channels, out_channels=self.planes, kernel_size=(kernel_size),
                               stride=1, padding=0, bias=False,)
            self.parallel_conv.append(sep_conv)

        self.bn1 = nn.BatchNorm1d(num_features=self.planes)
        self.relu = nn.ReLU(inplace=False)
        self.conv1 = nn.Conv1d(in_channels=self.planes, out_channels=self.planes, kernel_size=fixed_kernel_size,
                               stride=2, padding=2, bias=False)
        self.block = self._make_resnet_layer(kernel_size=fixed_kernel_size, stride=1, padding=fixed_kernel_size//2)
        self.bn2 = nn.BatchNorm1d(num_features=self.planes)
        self.avgpool = nn.AvgPool1d(kernel_size=6, stride=6, padding=2)
        self.rnn = Mamba(
                        # This module uses roughly 3 * expand * d_model^2 parameters
                        d_model=self.in_channels, # Model dimension d_model
                        d_state=128,  # SSM state expansion factor
                        d_conv=4,    # Local convolution width
                        expand=2,    # Block expansion factor
                    )
        # self.rnn = nn.GRU(input_size=self.in_channels, hidden_size=128, num_layers=1, bidirectional=True)
        self.fc = nn.Linear(in_features=424, out_features=num_classes)
        self.rnn1= Mamba(
                        # This module uses roughly 3 * expand * d_model^2 parameters
                        d_model=156, # Model dimension d_model
                        d_state=156,  # SSM state expansion factor
                        d_conv=4,    # Local convolution width
                        expand=2,    # Block expansion factor
                    )
        # self.rnn1 = nn.GRU(input_size=156, hidden_size=156, num_layers=1, bidirectional=True)

    def _make_resnet_layer(self, kernel_size, stride, blocks=9, padding=0):
        layers = []
        downsample = None
        base_width = self.planes

        for i in range(blocks
                    nn.MaxPool1d(kernel_size=2, stride=2, padding=0)
                )
            layers.append(ResNet_1D_Block(in_channels=self.planes, out_channels=self.planes, kernel_size=kernel_size,
                                       stride=stride, padding=padding, downsampling=downsampling))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = torch.transpose(x, 1, 2)
        out_sep = []

        for i in range(len(self.kernels)):
            sep = self.parallel_conv[i](x)
            out_sep.append(sep)

        out = torch.cat(out_sep, dim=2)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv1(out)  

        out = self.block(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.avgpool(out)  
        
        out = out.reshape(out.shape[0], -1)  

        rnn_out, _ = self.rnn(x.permute(0,2, 1))
        new_rnn_h = rnn_out[:, -1, :]  

        new_out = torch.cat([out, new_rnn_h], dim=1)  
        result = self.fc(new_out)  

        return result


In [10]:
from model import CustomModel
model = CustomModel() # EEGNet(kernels=[1,5,10,20]) #

In [11]:
model(torch.rand(33, 10000, 8)).size()

torch.Size([33, 6])

In [16]:
import torchmetrics
class HMSLitModule(pl.LightningModule):
    def __init__(self, 
                 model, 
                 loss = KLDivLossWithLogits(),
                 learning_rate_scheduler = None, 
                 weight_decay = 1e-6, # 1e-6
                 max_epoch = 10, 
                #  log_tool = "wandb",
                 cfg =None,
                 **kwargs):
        super().__init__()
        self.model = model
        self.loss = loss

        # if self.log_tool  =="wandb":
        #     wandb.init(project='hms', config=dict(cfg))


    def training_step(self, batch, batch_idx):
        # x, y = batch["data"], batch["target"]
        x, y = batch
        y_hat = self.model(x)
        loss = self.loss(y_hat, y)
        self.log('loss', loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        # x, y = batch["data"], batch["target"]
        x, y = batch
        y_hat = self.model(x)
        loss = self.loss(y_hat, y)
        self.log('val_loss', loss, on_epoch= True)
        return loss

    # def validation_epoch_end(self, batch, outs):
    #     # outs is a list of whatever you returned in `validation_step`
    #     loss = torch.stack(outs).mean()
    #     self.log("val_loss", loss)
    # def on_train_epoch_end(self):
        # wandb.log({"train_loss": loss})
    #     self.train_acc.reset()

    # def on_validation_epoch_end(self, outputs):
    #     self.log('valid_acc_epoch', self.valid_acc.compute())
    #     self.valid_acc.reset()
    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.model.parameters(), lr=5e-4)
        scheduler = LinearWarmupCosineAnnealingLR(optimizer, warmup_epochs=3, max_epochs=50)
        sch_config = {
            "scheduler": scheduler,
            "interval": "step",
            "name": "my_little_scheduler",
	    }
        return [optimizer], [sch_config]

In [17]:
train_dataset[1]

(array([[-1.0734377 ,  6.114375  , -3.79      , ..., -1.0625    ,
         -2.5215626 ,  4.0965624 ],
        [-1.1637497 ,  6.4015627 , -3.6490622 , ..., -1.0565624 ,
         -1.7603121 ,  3.8553123 ],
        [-0.27875042,  6.2737494 , -2.7575006 , ..., -1.1981249 ,
         -2.059062  ,  3.8156252 ],
        ...,
        [-2.5265622 ,  1.4449997 , -4.10625   , ..., -1.5796874 ,
         -1.4046874 , -0.3303125 ],
        [-2.85375   ,  1.9796875 , -3.814375  , ..., -1.1553125 ,
         -1.6774998 ,  0.45656252],
        [-2.5299997 ,  1.69875   , -4.0409374 , ..., -1.0615625 ,
         -1.265625  , -0.08374977]], dtype=float32),
 tensor([0.0000, 0.3333, 0.0000, 0.0000, 0.0000, 0.6667]))

In [18]:
from lightning.pytorch.loggers import WandbLogger
wandb_logger = WandbLogger(log_model="all")
model.to(device)
AVAIL_GPUS = min(1, torch.cuda.device_count())
trainer = pl.Trainer(callbacks=[checkpoint_callback], logger=wandb_logger, accelerator='gpu', devices=1)
hms_module = HMSLitModule(model = model.cuda(), cfg= cfg)
trainer.fit(
    hms_module, 
    train_dataloaders=train_loader, 
    val_dataloaders=val_loader,)




GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/ubuntu/.cache/pypoetry/virtualenvs/hmc-snuh-QMWTtYeW-py3.10/lib/python3.10/site-packages/pytorch_lightning/loops/utilities.py:94: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
You are using a CUDA device ('NVIDIA H100 80GB HBM3') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/home/ubuntu/.cache/pypoetry/virtualenvs/hmc-snuh-QMWTtYeW-py3.10/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:613: Checkpoint directory checkpoints/ exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES

Epoch 0:  14%|█▍        | 50/348 [02:29<14:50,  2.99s/it, loss=1.31]       

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: eru. Use `wandb login --relogin` to force relogin


Epoch 9:  78%|███████▊  | 272/348 [18:09<05:04,  4.01s/it, loss=0.748, v_num=5k4u]

In [ ]:
train_dataset[4]

(array([[ 3.1546876 , -2.8556252 ,  3.41125   , ..., -2.00875   ,
          4.171875  , -3.6334374 ],
        [ 1.6018751 , -3.4728124 ,  1.9696875 , ..., -0.13656247,
          2.5865624 , -1.243125  ],
        [ 0.71343756, -3.2353125 ,  1.09375   , ...,  0.11843753,
          2.5190625 , -1.3634374 ],
        ...,
        [ 3.9434376 , -3.8453124 ,  3.7371874 , ..., -1.1993749 ,
          4.0453124 , -2.4271874 ],
        [ 4.4621873 , -3.57625   ,  4.231875  , ..., -1.4131249 ,
          4.385937  , -2.69875   ],
        [ 2.479375  , -3.753125  ,  1.6125    , ...,  0.541875  ,
          2.9896874 , -0.429375  ]], dtype=float32),
 tensor([ 0.,  0.,  0.,  0.,  0., 30.]))